In [154]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange


import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed

from flygym.mujoco import Parameters


from nmf_grooming import (
    NeuromechflyGrooming,
    all_groom_dofs,
    load_grooming_data,
    plot_state_and_contacts,
)
from scipy.signal import find_peaks, medfilt
from IPython.display import Video

from flygym.mujoco.examples.rule_based_controller import PreprogrammedSteps

In [155]:
run_time = 1

sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

In [156]:
target_num_steps = int(run_time / sim_params.timestep)


In [157]:
nmf = flygym.mujoco.NeuroMechFly(Parameters(
    enable_adhesion=True, 
    draw_adhesion=True,
    render_camera="Animat/camera_left"))

preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

actuated_joints = preprogrammed_steps.dofs_per_leg

standing_action = []

for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1]))
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))
    
last_position = standing_action

standing_action = {'joints': standing_action, "adhesion": np.zeros(len(legs))}

for i in range(int(0.2//nmf.timestep)):
    nmf.step(standing_action)



In [158]:
foreleg_ids = np.zeros(target_num_steps)
print(swing_periods["RM"][1])
middle_stance_ids = np.linspace(swing_periods["RM"][1], swing_periods["RM"][1], target_num_steps)
hind_swings_ids = np.linspace(swing_periods["RM"][1], 1/4*np.pi, target_num_steps)

adhesion_action = np.array([0.0 for leg in legs])

all_joint_angles = np.ones((target_num_steps, len(last_position)))

for i in range(len(all_joint_angles[0])):
    if i == 7 :
        all_joint_angles[:,i] = np.linspace(last_position[7], -1/16*np.pi, target_num_steps)
    elif i == 8:
        all_joint_angles[:,i] = np.linspace(last_position[8], 1/4*np.pi, target_num_steps)
    elif i == 12:
        all_joint_angles[:,i] = np.linspace(last_position[12], 1/3*np.pi, target_num_steps)
    elif i == 13:
        all_joint_angles[:,i] = np.linspace(last_position[13], 0, target_num_steps)
    elif i == 16:
        all_joint_angles[:,i] = np.linspace(last_position[16], -1/8*np.pi, target_num_steps)
    elif i == 19:
        all_joint_angles[:,i] = np.linspace(last_position[19], 3/4*np.pi, target_num_steps)
    elif i == 28:
        all_joint_angles[:,i] = np.linspace(last_position[28], -1/16*np.pi, target_num_steps)
    elif i == 29:
        all_joint_angles[:,i] = np.linspace(last_position[29], -1/4*np.pi, target_num_steps)
    elif i == 33:
        all_joint_angles[:,i] = np.linspace(last_position[33], 1/3*np.pi, target_num_steps)
    elif i == 34:
        all_joint_angles[:,i] = np.linspace(last_position[34], 0, target_num_steps)
    elif i == 37:
        all_joint_angles[:,i] = np.linspace(last_position[37], 1/8*np.pi, target_num_steps)
    elif i == 40:
        all_joint_angles[:,i] = np.linspace(last_position[40], 3/4*np.pi, target_num_steps)
    else:
        all_joint_angles[:,i] = np.linspace(last_position[i], last_position[i], target_num_steps)


for i in range(target_num_steps):
    joint_angles = all_joint_angles[i,:]
    action = {'joints': np.array(joint_angles), "adhesion": adhesion_action}

    nmf.step(action)
    nmf.render()

adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])
action = {'joints': all_joint_angles[-1,:], "adhesion": adhesion_action}
nmf.step(action)
nmf.render()


nmf.save_video("./outputs/test_b.mp4")

last_joint_angles = all_joint_angles[-1]

2.234021442552742


In [159]:
Video("./outputs/test_b.mp4")

In [78]:
all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0])))

for i in range(len(all_joint_angles[0])):
    if i == 5:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[5], 1/3*np.pi, target_num_steps)
    elif i == 6 :
        all_joint_angles[:,i] = np.linspace(last_joint_angles[6], 0, target_num_steps)
    elif i == 12:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[12], 1/3*np.pi, target_num_steps)
    elif i == 13:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[13], 0, target_num_steps)
    elif i == 26:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[26], 1/3*np.pi, target_num_steps)
    elif i == 27:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[27], 0, target_num_steps)
    elif i == 33:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[33], 1/3*np.pi, target_num_steps)
    elif i == 34:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[34], 0, target_num_steps)
    else:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[i], last_joint_angles[i], target_num_steps)



adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in range(target_num_steps):
    joint_angles = all_joint_angles[i,:]
    action = {'joints': joint_angles, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
nmf.save_video("./outputs/b.mp4")

In [ ]:
Video("./outputs/b.mp4")